In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import random
from time import sleep

In [2]:
h1 = {
'Cookie': 'bid=1ADu4pNSAKc; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1700053849%2C%22https%3A%2F%2Fwww.google.com%2F%22%5D; _pk_id.100001.4cf6=f4c8ada49137d86c.1700053849.; _pk_ses.100001.4cf6=1; __utma=30149280.1448304485.1700053850.1700053850.1700053850.1; __utmb=30149280.0.10.1700053850; __utmc=30149280; __utmz=30149280.1700053850.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utma=223695111.1172310301.1700053850.1700053850.1700053850.1; __utmc=223695111; __utmz=223695111.1700053850.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); ap_v=0,6.0; __yadk_uid=Lxc1V2RNH8sxH0UcXFowuXqWXReWgu92; _ga=GA1.2.1779191722.1700054711; _gid=GA1.2.205436017.1700054711; ll="108090"; _ga_Y4GN1R87RG=GS1.1.1700054710.1.0.1700054716.0.0.0; __utmt=1; _vwo_uuid_v2=D52BC5697628080D9B6777173F7D54E0D|7c1dafdce1bc1807a6f799731203a0ef; dbcl2="275921482:DFDGuKKqB5g"; ck=Oljv; frodotk_db="44fc9912780cdcf0cef5cb37682696d0"; push_noty_num=0; push_doumail_num=0; __utmb=223695111.5.10.1700053850',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
'Accept-Encoding': 'gzip, deflate',
'Host': 'movie.douban.com',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15',
'Accept-Language': 'zh-CN,zh-Hans;q=0.9',
'Referer': 'https://movie.douban.com/subject/35267224/?from=showing',
'Connection': 'keep-alive'
}

In [3]:
def trans_star(v_str):
    v_str = v_str[0]
    if v_str == 'allstar10':
        return '1星'
    elif v_str == 'allstar20':
        return '2星'
    elif v_str == 'allstar30':
        return '3星'
    elif v_str == 'allstar40':
        return '4星'
    elif v_str == 'allstar50':
        return '5星'
    else:
        return '未知'

In [6]:
def get_short(v_movie_id):
    for page in range(1, max_page + 1):  
        requests.packages.urllib3.disable_warnings()
        url = 'https://movie.douban.com/subject/{}/comments?start={}&limit=20&status=P&sort=new_score'.format(
            v_movie_id, (page - 1) * 20)
        response = requests.get(url, headers=h1, verify=False)
        print(response.status_code)
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.find_all('div', {'class': 'comment'})
        print('开始爬取第{}页，共{}条评论'.format(page, len(reviews)))
        sleep(random.uniform(1, 2))
        user_name_list = []  
        star_list = []  
        time_list = []  
        ip_list = []  
        vote_list = []  
        content_list = []  
        for review in reviews:
            user_name = review.find('span', {'class': 'comment-info'}).find('a').text
            user_name_list.append(user_name)
            star = review.find('span', {'class': 'comment-info'}).find_all('span')[1].get('class')
            star = trans_star(star)
            star_list.append(star)
            time2 = review.find('span', {'class': 'comment-time'}).text.strip()
            print('评论时间：', time2)
            time_list.append(time2)
            ip = review.find('span', {'class': 'comment-location'}).text
            ip_list.append(ip)
            vote = review.find('span', {'class': 'votes vote-count'}).text
            vote_list.append(vote)
            content = review.find('span', {'class': 'short'}).text
            content = content.replace(',', '，').replace(' ', '').replace('\n', '').replace('\t', '').replace('\r', '')
            content_list.append(content)
        df = pd.DataFrame(
            {
                '页码': page,
                '评论者昵称': user_name_list,
                '评论星级': star_list,
                '评论时间': time_list,
                '评论者IP属地': ip_list,
                '有用数': vote_list,
                '评论内容': content_list,
            }
        )
        if os.path.exists(result_file):
            header = False
        else:
            header = True
        df.to_csv(result_file, mode='a+', header=header, index=False, encoding='utf_8_sig')
        print('文件保存成功：', result_file)

In [7]:
movie_id = '26258779'
max_page = 30  
result_file = '豆瓣短评_{}_前{}页.csv'.format(movie_id, max_page)
if os.path.exists(result_file):
    os.remove(result_file)
    print('结果文件存在，已删除: {}'.format(result_file))
get_short(movie_id)

200
开始爬取第1页，共20条评论
评论时间： 2023-05-05 04:19:59
评论时间： 2023-05-03 18:18:08
评论时间： 2023-05-03 17:57:58
评论时间： 2023-05-06 06:26:37
评论时间： 2023-05-03 18:10:56
评论时间： 2023-05-03 18:24:48
评论时间： 2023-05-03 17:44:41
评论时间： 2023-05-07 01:09:58
评论时间： 2023-05-07 14:01:30
评论时间： 2023-04-30 00:16:29
评论时间： 2023-05-03 10:07:16
评论时间： 2023-05-05 18:49:14
评论时间： 2023-05-03 10:04:34
评论时间： 2023-05-06 06:50:43
评论时间： 2023-05-03 18:41:17
评论时间： 2023-05-05 16:11:13
评论时间： 2023-09-04 00:14:47
评论时间： 2023-05-03 20:00:33
评论时间： 2023-05-05 23:56:00
评论时间： 2023-05-03 18:12:06
文件保存成功： 豆瓣短评_26258779_前30页.csv
200
开始爬取第2页，共20条评论
评论时间： 2023-05-04 09:45:56
评论时间： 2023-05-05 21:07:28
评论时间： 2023-05-05 18:57:25
评论时间： 2023-05-06 16:56:17
评论时间： 2023-05-05 20:15:05
评论时间： 2023-05-03 18:02:36
评论时间： 2023-05-05 12:52:22
评论时间： 2023-05-15 08:54:32
评论时间： 2023-05-06 17:03:08
评论时间： 2023-05-05 17:17:36
评论时间： 2023-05-03 18:03:36
评论时间： 2023-05-03 22:18:58
评论时间： 2023-05-06 21:45:01
评论时间： 2023-05-07 17:35:06
评论时间： 2023-05-03 13:21:02
评论时间： 2023-05-19 09:0

200
开始爬取第16页，共20条评论
评论时间： 2023-05-06 08:31:43
评论时间： 2023-05-18 08:53:28
评论时间： 2023-06-04 00:05:49
评论时间： 2023-07-29 16:25:51
评论时间： 2023-05-09 18:57:55
评论时间： 2023-05-18 21:14:39
评论时间： 2023-07-22 17:40:24
评论时间： 2023-05-07 02:04:33
评论时间： 2023-05-05 18:07:01
评论时间： 2023-07-21 18:43:18
评论时间： 2023-07-09 04:04:05
评论时间： 2023-05-05 19:17:33
评论时间： 2023-05-14 18:23:53
评论时间： 2023-05-05 02:59:43
评论时间： 2023-05-14 11:13:18
评论时间： 2023-06-13 23:20:19
评论时间： 2023-05-05 17:34:11
评论时间： 2023-05-09 22:34:19
评论时间： 2023-05-20 11:12:08
评论时间： 2023-05-06 16:27:00
文件保存成功： 豆瓣短评_26258779_前30页.csv
200
开始爬取第17页，共20条评论
评论时间： 2023-05-05 23:23:08
评论时间： 2023-05-26 21:45:44
评论时间： 2023-05-08 16:24:26
评论时间： 2023-05-11 20:41:17
评论时间： 2023-05-07 22:09:14
评论时间： 2023-05-07 16:41:40
评论时间： 2023-05-03 20:06:28
评论时间： 2023-05-08 17:59:40
评论时间： 2023-07-23 18:42:39
评论时间： 2023-05-07 13:43:21
评论时间： 2023-07-07 18:13:23
评论时间： 2023-05-05 18:57:45
评论时间： 2023-07-17 12:55:38
评论时间： 2023-05-07 02:59:48
评论时间： 2023-05-05 19:58:06
评论时间： 2023-05-06 00